<a href="https://colab.research.google.com/github/zhannatoleubek-png/special-okx-chainsaw/blob/main/long-s.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# -----------------------------
# MAIN LOOP (Momentum Flow Pro, все USDT пары)
# -----------------------------
!pip install --quiet ccxt pandas numpy ta

import ccxt
import pandas as pd
import numpy as np
import time
from datetime import datetime, timezone

# -----------------------------
# CONFIG — настраиваемые параметры
# -----------------------------
CONFIG = {
    "exchange_id": "okx",
    "timeframes": {"1m":"1m","3m":"3m","5m":"5m","30m":"30m","1h":"1h"},
    "limit": 500,
    "analysis_interval_sec": 60,
    "sl_pct": 0.003,
    "tp_pct": 0.008,
    "volume_spike_mult": 1.5,
    "max_distance_from_ema50_pct": 0.01,
    "rsi_overbought": 70,
    "rsi_entry_level": 50,
    "heikin_ashi": True,
    "telegram": {
        "enabled": False,
        "bot_token": "<8294892098:AAFX0Zzq9yN1on6UlID8f7vzif4dWR_7uWs>",
        "chat_id": "<381202205>"
    }
}

# -----------------------------
# UTILS & INDICATORS
# -----------------------------
def init_exchange(cfg):
    ex_class = getattr(ccxt, cfg["exchange_id"])
    ex = ex_class({'enableRateLimit': True})
    ex.load_markets()
    return ex

def fetch_ohlcv_df(exchange, symbol, timeframe, limit=500):
    ohlcv = exchange.fetch_ohlcv(symbol, timeframe=timeframe, limit=limit)
    df = pd.DataFrame(ohlcv, columns=['ts','open','high','low','close','volume'])
    df['datetime'] = pd.to_datetime(df['ts'], unit='ms', utc=True)
    df.set_index('datetime', inplace=True)
    return df

def heikin_ashi(df):
    ha = df.copy()
    ha['ha_close'] = (ha['open'] + ha['high'] + ha['low'] + ha['close'])/4
    ha['ha_open'] = 0.0
    ha.iloc[0, ha.columns.get_loc('ha_open')] = (ha.iloc[0]['open'] + ha.iloc[0]['close'])/2
    for i in range(1, len(ha)):
        ha.iat[i, ha.columns.get_loc('ha_open')] = (ha.iat[i-1, ha.columns.get_loc('ha_open')] + ha.iat[i-1, ha.columns.get_loc('ha_close')])/2
    ha['ha_high'] = ha[['ha_open','ha_close','high']].max(axis=1)
    ha['ha_low'] = ha[['ha_open','ha_close','low']].min(axis=1)
    return ha[['ha_open','ha_high','ha_low','ha_close']]

def ema(series, period):
    return series.ewm(span=period, adjust=False).mean()

def add_vwap_daily(df):
    g = df.copy()
    g['tp'] = (g['high'] + g['low'] + g['close'])/3
    g = g.groupby(g.index.date).apply(lambda gp: gp.assign(vwap=(gp['tp']*gp['volume']).cumsum()/gp['volume'].cumsum()))
    return g['vwap']

def rsi(series, period=14):
    delta = series.diff()
    up = delta.clip(lower=0)
    down = -1 * delta.clip(upper=0)
    ma_up = up.ewm(alpha=1/period, adjust=False).mean()
    ma_down = down.ewm(alpha=1/period, adjust=False).mean()
    rs = ma_up / ma_down
    return 100 - (100 / (1 + rs))

def wavetrend(df, n1=10, n2=21):
    price = (df['high'] + df['low'] + df['close'])/3
    esa = price.ewm(span=n1, adjust=False).mean()
    d = (price - esa).abs().ewm(span=n1, adjust=False).mean()
    ci = (price - esa) / (0.015 * d)
    wt1 = ci.ewm(span=n2, adjust=False).mean()
    wt2 = wt1.ewm(span=4, adjust=False).mean()
    return wt1, wt2

def mfi(df, period=14):
    typical = (df['high'] + df['low'] + df['close'])/3
    money = typical * df['volume']
    positive = (typical > typical.shift(1)).astype(int) * money
    negative = (typical < typical.shift(1)).astype(int) * money
    pos_sum = positive.rolling(period).sum()
    neg_sum = negative.rolling(period).sum()
    mfr = pos_sum / (neg_sum.replace(0, np.nan))
    return 100 - (100 / (1 + mfr)).fillna(50)

def detect_green_dot(df):
    wt1, wt2 = wavetrend(df)
    df['wt1'] = wt1
    df['wt2'] = wt2
    df['wt_cross_up'] = (df['wt1'] > df['wt2']) & (df['wt1'].shift(1) <= df['wt2'].shift(1))
    df['wt_rising'] = df['wt1'].diff() > 0
    df['mfi'] = mfi(df)
    df['rsi'] = rsi(df['close'])
    df['green_dot'] = df['wt_cross_up'] & (df['mfi'] > df['mfi'].shift(1)) & (df['rsi'] > 45)
    return df

# -----------------------------
# SIGNAL LOGIC
# -----------------------------
def analyze_pair(exchange, symbol, cfg):
    dfs = {}
    now = None
    for label, tf in cfg['timeframes'].items():
        try:
            df = fetch_ohlcv_df(exchange, symbol, tf, limit=cfg['limit'])
        except:
            continue
        dfs[label] = df
        if now is None and len(df):
            now = df.index[-1]
    if '1h' not in dfs or '30m' not in dfs:
        return {"signal":None,"reason":"Missing H1 or M30 data","time":now.isoformat() if now else None}
    h1 = dfs['1h']
    m30 = dfs['30m']
    h1['ema50'] = ema(h1['close'],50)
    h1['ema200'] = ema(h1['close'],200)
    m30['ema50'] = ema(m30['close'],50)
    m30['ema200'] = ema(m30['close'],200)
    for tf_df in [h1,m30]:
        tf_df['vwap'] = add_vwap_daily(tf_df)
    bullish_h1 = (h1['ema50'].iloc[-1] > h1['ema200'].iloc[-1]) and (h1['close'].iloc[-1] > h1['vwap'].iloc[-1])
    bullish_m30 = (m30['ema50'].iloc[-1] > m30['ema200'].iloc[-1]) and (m30['close'].iloc[-1] > m30['vwap'].iloc[-1])
    if not (bullish_h1 and bullish_m30):
        return {"signal": None, "reason": "Context not bullish on H1/M30", "time": now.isoformat() if now else None}
    for label in ['1m','3m','5m']:
        df = dfs.get(label)
        if df is None: continue
        if cfg['heikin_ashi']:
            ha = heikin_ashi(df)
            df = df.join(ha, how='left')
            df['use_close'] = df['ha_close']
            df['use_high'] = df['ha_high']
            df['use_low'] = df['ha_low']
        else:
            df['use_close'] = df['close']
            df['use_high'] = df['high']
            df['use_low'] = df['low']
        df['ema9'] = ema(df['use_close'],9)
        df['ema50'] = ema(df['use_close'],50)
        df['rsi'] = rsi(df['use_close'],14)
        df['vwap'] = add_vwap_daily(df)
        df = detect_green_dot(df)
        df['vol_mean10'] = df['volume'].rolling(10).mean()
        df['vol_spike'] = df['volume'] > (df['vol_mean10'] * cfg['volume_spike_mult'])
        last_price = df['use_close'].iloc[-1]
        last_ema50 = df['ema50'].iloc[-1]
        dist_pct = abs(last_price - last_ema50)/last_ema50 if last_ema50 else 0.0
        cond_green_dot = df['green_dot'].iloc[-1]
        cond_rsi = df['rsi'].iloc[-1] > cfg['rsi_entry_level']
        cond_vol = df['vol_spike'].iloc[-1]
        cond_close_above_ema9 = df['use_close'].iloc[-1] > df['ema9'].iloc[-1]
        cond_dist_ok = dist_pct <= cfg['max_distance_from_ema50_pct']
        cond_not_overbought = df['rsi'].iloc[-1] < cfg['rsi_overbought']
        prev_high = df['use_high'].iloc[-2]
        breakout = df['use_close'].iloc[-1] > prev_high
        strong_signal = all([cond_green_dot,cond_rsi,cond_vol,cond_close_above_ema9,cond_dist_ok,cond_not_overbought,breakout])
        weak_signal = all([cond_green_dot,cond_rsi,cond_close_above_ema9,cond_dist_ok,cond_not_overbought,breakout])
        if strong_signal:
            entry_price = df['use_close'].iloc[-1]
            sl = df['use_low'].iloc[-2]
            if sl >= entry_price: sl = entry_price*(1-cfg['sl_pct'])
            tp = entry_price*(1+cfg['tp_pct'])
            return {"signal":"STRONG_LONG","tf":label,"entry":float(entry_price),"sl":float(sl),"tp":float(tp),"reason":"GreenDot + RSI+Volume+Breakout + H1/M30 bullish","time":now.isoformat()}
        elif weak_signal:
            entry_price = df['use_close'].iloc[-1]
            sl = entry_price*(1-cfg['sl_pct'])
            tp = entry_price*(1+cfg['tp_pct'])
            return {"signal":"LONG","tf":label,"entry":float(entry_price),"sl":float(sl),"tp":float(tp),"reason":"GreenDot + RSI + Breakout (no vol spike)","time":now.isoformat()}
    return {"signal":None,"reason":"No entry conditions met","time":now.isoformat()}

# -----------------------------
# MAIN LOOP
# -----------------------------
def main_loop(cfg):
    exchange = init_exchange(cfg)
    usdt_symbols = [s for s in exchange.markets if 'USDT' in s]
    print(f"Exchange loaded. {len(usdt_symbols)} USDT symbols found.")
    try:
        while True:
            t0 = time.time()
            for symbol in usdt_symbols:
                try:
                    res = analyze_pair(exchange, symbol, cfg)
                    now_local = datetime.now(timezone.utc).astimezone().isoformat()
                    if res.get("signal"):
                        print(f"[{now_local}] {symbol} SIGNAL: {res['signal']} | TF: {res.get('tf')} | ENTRY: {res['entry']:.2f} SL: {res['sl']:.2f} TP: {res['tp']:.2f} | {res['reason']}")
                    else:
                        print(f"[{now_local}] {symbol} No signal. Reason: {res['reason']}")
                except Exception as e:
                    print(f"Error analyzing {symbol}: {e}")
            elapsed = time.time() - t0
            time.sleep(max(1, cfg['analysis_interval_sec'] - elapsed))
    except KeyboardInterrupt:
        print("Stopped by user.")

# -----------------------------
# Запуск
# -----------------------------
if __name__ == "__main__":
    main_loop(CONFIG)


Выходные данные были обрезаны до нескольких последних строк (5000).
[2025-10-19T12:44:36.996891+00:00] NMR/USDT No signal. Reason: Context not bullish on H1/M30
[2025-10-19T12:44:38.270098+00:00] NOT/USDT No signal. Reason: Context not bullish on H1/M30
[2025-10-19T12:44:39.530933+00:00] OAS/USDT No signal. Reason: Context not bullish on H1/M30
[2025-10-19T12:44:40.791887+00:00] OKSOL/USDT No signal. Reason: Context not bullish on H1/M30
[2025-10-19T12:44:42.054716+00:00] OL/USDT No signal. Reason: Context not bullish on H1/M30
[2025-10-19T12:44:43.282807+00:00] OM/USDT No signal. Reason: Context not bullish on H1/M30
[2025-10-19T12:44:44.529966+00:00] OMI/USDT No signal. Reason: Context not bullish on H1/M30
[2025-10-19T12:44:45.792288+00:00] ONDO/USDT No signal. Reason: Context not bullish on H1/M30
[2025-10-19T12:44:47.024216+00:00] ONE/USDT No signal. Reason: Context not bullish on H1/M30
[2025-10-19T12:44:48.260239+00:00] ONT/USDT No signal. Reason: Context not bullish on H1/M30
[